### BEGINING

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


**COLAB ENVIROMENT VARIABLE MANAGER**

In [3]:
# for loading environment variable
!pip install python-dotenv

In [4]:
# Set your GitHub personal access token
from dotenv import load_dotenv
import os

# Load the environment variables from the file on Google Drive
load_dotenv('/content/drive/My Drive/file.env')

token = os.getenv('GIT_TOKEN')

**CLONE THE REPOSITORY**

In [5]:
# Clone the repository using the token
!git clone https://{token}@github.com/omogbolahan94/Vector-Database.git

Cloning into 'Vector-Database'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
Receiving objects: 100% (6/6), done.
Resolving deltas: 100% (1/1), done.
remote: Total 6 (delta 1), reused 5 (delta 0), pack-reused 0


**COPY THIS NOTEBOOK TO THE REPOSITORY**

In [13]:
!pwd

/content


In [14]:
!ls

drive  sample_data  Vector-Database


In [15]:
!cp '/content/drive/MyDrive/Colab Notebooks/LLM_vector_database.ipynb' /content/Vector-Database/

**CONFIGURE GIT**

In [16]:
!git config --global user.name "omogbolahan94"
!git config --global user.email "gabrielomogbolahan1@gmail.com"

**NAVIGATE TO THE CLONED REPOSITORY**

In [17]:
%cd '/content/Vector-Database/'

/content/Vector-Database


In [18]:
!ls

LLM_vector_database.ipynb  README.md


### MAIN CODE

**PREPARE THE DATASET**

In [26]:
import json

with open('/content/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

**CREATE EMBEDDINGS WITH PRE-TRAISNED MODEL**

In [32]:
# This specific version works. Latest version may not work

!pip install sentence_transformers==2.7.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.2 MB/s eta 0:00:00


In [33]:
# This is a new library compared to the previous modules.
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [34]:
sample_sentence = "This is a simple sentence"

# create embeddings for the sentence above:
embeded_sample = model.encode(sample_sentence)
print(f"length of embeded sample: {len(embeded_sample)}'\n\n'")
embeded_sample[0:5]

length of embeded sample: 768'

'


array([ 0.00444873, -0.07613142, -0.00037745,  0.00752527, -0.03809793],
      dtype=float32)

In [35]:
# created the dense vector for each text in the document using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

**CONNECT TO RUNNING ELASTICSEARCH**

In [ ]:
# from elasticsearch import Elasticsearch

In [ ]:
# es_client = Elasticsearch('http://localhost:9200')

# es_client.info()

### CHECK STATUS, ADD AND COMMIT CHANGES

In [29]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [30]:
!git add .
!git commit -m "added the raw document data"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [31]:
!git push origin main

Everything up-to-date
